In [ ]:
import sys
if "google.colab" in sys.modules:
    print("Colab Detected")
    import tensorflow as tf
    gpus = tf.config.list_physical_devices("GPU")
    if not gpus:
        raise RuntimeError("Nessuna GPU trovata.Controlla di aver selezionato il runtime giusto.")
    else:
        print(f"Trovate {len(gpus)} GPU:\n{gpus}")

    !git clone https://github.com/AtomicDonuts/Progetto_Computings.git
    %cd Progetto_Computings/
    !pip install -q -r requirements.txt
    !python3 fits_import/fits2csv.py

    sys.path.append("imports/")
    import custom_variables as custom_paths
    import ann_models as ann
else:
    print("Local Machine Detected")
    sys.path.append("../imports/")
    import custom_variables as custom_paths
    import ann_models as ann

In [ ]:
import numpy as np
import pandas as pd
import keras
import keras_tuner as kt
from sklearn.model_selection import StratifiedKFold

In [ ]:
df = pd.read_csv(custom_paths.csv_path)
df = df[(df["CLASS_GENERIC"] == "AGN") | (df["CLASS_GENERIC"] == "Pulsar")]
print(f"Dataset Dimentions: {len(df)}")

df["PowerLaw"] = np.where(df["SpectrumType"] == "PowerLaw",1,0,)
df["LogParabola"] = np.where(df["SpectrumType"] == "LogParabola",1,0,)
df["PLSuperExpCutoff"] = np.where(df["SpectrumType"] == "PLSuperExpCutoff",1,0,)

norm_cols = ["GLAT", "PowerLaw","LogParabola","PLSuperExpCutoff" ,"Variability_Index"]
input_datas = df[norm_cols].to_numpy()

is_agn = df["CLASS_GENERIC"].to_numpy() == "AGN"
is_psr = df["CLASS_GENERIC"].to_numpy() == "Pulsar"
labels = np.zeros((len(df)), dtype=int)
labels[is_agn] = 0
labels[is_psr] = 1

In [ ]:
skf = StratifiedKFold(n_splits=10, shuffle=True)
a,b = next(skf.split(input_datas, labels))

In [ ]:
train_inp = input_datas[a]
train_lab = labels[a]
val_inp = input_datas[b]
val_lab = labels[b]

In [ ]:
tuner = kt.Hyperband(
    ann.hp_model,
    objective="accuracy",
    max_epochs=100,
    overwrite=True,
    directory="my_dir",
    project_name="prova",
)

stop_early = keras.callbacks.EarlyStopping(monitor="accuracy", patience=5)

In [ ]:
tuner.search(
    train_inp, train_lab, epochs=50, validation_data=(val_inp,val_lab), callbacks=[stop_early]
)